In [1]:
import os
import torch

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/iberlef/IberLEF2023

/content/gdrive/MyDrive/iberlef/IberLEF2023


In [3]:
! git status

Refresh index: 100% (11/11), done.

No commits yet

Changes to be committed:
  (use "git rm --cached <file>..." to unstage)
	new file:   .gitignore
	new file:   README.md
	new file:   data/full_data/cleaned/cleaned_full_data.csv
	new file:   data/full_data/full_data.csv
	new file:   data/practise_data/cleaned/cleaned_development_test.csv
	new file:   data/practise_data/cleaned/cleaned_development_train.csv
	new file:   data/practise_data/development_test.csv
	new file:   data/practise_data/development_train.csv
	new file:   data/synthetic_data/synthetic_data.csv
	new file:   preprocess.ipynb
	new file:   req.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	politibeto_pretrain.ipynb



Pretraining PolitiBETO on this year's IberLEF shared task dataset that has been cleaned with [preprocess.ipynb](https://github.com/Abbywpy/IberLEF2023/blob/main/preprocess.ipynb).

In [33]:
import pandas as pd

## Preprocessing

In [42]:
if not os.path.exists("./data/full_data/cleaned/cleaned_full_data.csv"):
    df = pd.read_csv("./data/full_data/cleaned/cleaned_full_data.csv")
    df["cleaned_tweet"] = df["cleaned_tweet"].astype(str)
    df_tweets = pd.DataFrame(df["cleaned_tweet"])
    df_tweets.to_csv("./data/full_data/cleaned/cleaned_tweets_only.csv", index=False)

In [43]:
from datasets import load_dataset

# replace the path for according to the data you want to use
datafile = "cleaned_tweets_only.csv"
dataset = load_dataset('csv', data_files=os.path.join("./data/full_data/cleaned", datafile))

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 142.55it/s]
Generating train split: 0 examples [00:00, ? examples/s]/Users/michelle/Desktop/cl/ml4nlp2/iberlef/iberenv/lib/python3.8/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /Users/michelle/.cache/huggingface/datasets/csv/default-1efde2b9b06a2be3/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 17.10it/s]


In [44]:
""" this is only to test if the code works, don't use in actual training """
# from datasets import DatasetDict
#
# print(type(dataset))
# dataset = dataset["train"].select(range(10))
#
# dataset = DatasetDict({"train": dataset})
#
# print(type(dataset))

<class 'datasets.dataset_dict.DatasetDict'>
<class 'datasets.dataset_dict.DatasetDict'>


In [45]:
print(dataset["train"][0])

{'cleaned_tweet': '¡Feliz 28 de Febrero a Todas Las Andaluzas Y Andaluces! . Por Una Andalucía Donde El presente, Y El Futuro, sea Sinónimo de Derechos, de Igualdad Plena, de Trabajo Estable Y con Condiciones Dignas . Una Andalucía de Nuevas Oportunidades, de Esperanza.'}


In [46]:
from transformers import BertTokenizer

model_checkpoint = "nlp-cimat/politibeto"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["cleaned_tweet"], padding=True, truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [48]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained(model_checkpoint)

In [56]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
training_args = TrainingArguments(
    output_dir="./pretrained_politibeto/" + datafile[:-4].split("_",1)[1],
    overwrite_output_dir=True,
    num_train_epochs=3,  # hyperparameters the same as mentioned in Villa-Cueva et al. (2022)
    per_device_train_batch_size=128,
    learning_rate=5e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
)

In [57]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=1.506382902463277, metrics={'train_runtime': 38.2203, 'train_samples_per_second': 0.785, 'train_steps_per_second': 0.078, 'total_flos': 956179090800.0, 'train_loss': 1.506382902463277, 'epoch': 3.0})

In [59]:
trainer.save_model()